In [1]:
import langchain
import faiss
import numpy as np
import zhipuai
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
from tqdm import tqdm
import os
from pathlib import Path
import random
import re
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

In [2]:
zhipuai.api_key = "29211907a4374e6ec56e57e617486996.NPLv443cZVXLmP40"

In [97]:
DEFAULT_QA_GENERATE_PROMPT_TMPL = """\
Context information is below.

---------------------
{context_str}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Teacher/ Professor. Your task is to setup \
{num_questions_per_chunk} questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. Restrict the questions to the \
context information provided. The question is short answer question.
---------------------
The format is 
[question]your give question1
[question]you give question2
"""

def queryLLM(chunk, questionNum):
    query = DEFAULT_QA_GENERATE_PROMPT_TMPL.format(context_str=chunk, num_questions_per_chunk=str(questionNum))
    response = zhipuai.model_api.sse_invoke(
        model="chatglm_turbo",
        prompt=[
            #{"role": "user", "content": "#要求 /n 请将资料里的内容重写总结 /n #资料 /n "+chunks[0]},
            {"role": "user", "content": query},
        ],
        temperature=0,
        top_p=0.7,
        incremental=True
    )
    outstr = "" 
    for event in response.events():
        outstr = outstr+event.data
    return outstr

def generateQuestion(data):
    allQuestionDict = {}
    questionNum = 2
    for key in tqdm(data):
        signalQuestionList = []
        llmResponse = queryLLM(data[key], questionNum)
        result = str(llmResponse).strip().split("[question]")
        questions = [re.sub(r"^\d+[\).\s]", "", question).strip() for question in result]
        questions = [question for question in questions if len(question) > 0]
        print("new*******",questions,"\n")
        allQuestionDict[key] = questions
    return allQuestionDict

In [102]:
class chunksData():
    def __init__(self, fileName, chunkSize, chunkOverlap):
        [self.fileName, self.fileNameType] = fileName.split(".")
        self.chunkSize = chunkSize
        self.chunkOverlap = chunkOverlap
        self.startPos = []
        start = 0
        while start + self.chunkOverlap < self.chunkSize:
            self.startPos.append(start)
            start = start + self.chunkOverlap
        self.jsonNum = len(self.startPos)
        self.allChunksMap = []
        self.loadData()
    def loadData(self):
        for i in tqdm(range(self.jsonNum)):
            tmpFileName = self.fileName+"_"+str(self.chunkSize)+"_"+str(self.chunkOverlap)+"_"+str(i)+".json"
            if os.path.isfile(tmpFileName):
                self.allChunksMap.append(self.openData(tmpFileName))
            else:
                self.allChunksMap.append(self.dealData(self.fileName+"."+self.fileNameType, tmpFileName, self.startPos[i]))
    def openData(self, fileName):
        with open(fileName, 'r', encoding='utf-8') as file:
            chunksMap = json.load(file)
        return chunksMap
    def dealData(self, oriFileName, jsonFileName, start):
        chunksMap = {}
        with open(oriFileName, 'r') as file:
            fileContents = file.read()
        tmpStart = start
        end = start + self.chunkSize
        count = 0
        while(end<len(fileContents)):
            chunksMap[str(count)] = fileContents[tmpStart:end]
            count = count + 1
            tmpStart = tmpStart + self.chunkSize - self.chunkOverlap
            end = tmpStart + self.chunkSize
        with open(jsonFileName, 'w',encoding='utf-8') as file:
            json.dump(chunksMap, file)
        return chunksMap
    def splitTrainAndVal(self):
        if os.path.isfile("valKeys.json"):
            with open("valKeys.json", 'r', encoding='utf-8') as file:
                self.valKeys = json.load(file)
            with open("trainKeys.json", 'r', encoding='utf-8') as file:
                self.trainKeys = json.load(file)
        else:
            self.valKeys = []
            self.trainKeys = []
            for i in tqdm(range(self.jsonNum)):
                valKeys, trainKeys  = self.generateDataset(self.allChunksMap[i])
                prefix = str(i) + "_"
                self.valKeys.extend([prefix + s for s in valKeys])
                self.trainKeys.extend([prefix + s for s in trainKeys])
            valJson = json.dumps(self.valKeys)
            trainJson = json.dumps(self.trainKeys)
            with open("valKeys.json", 'w') as json_file:
                json_file.write(valJson)
            with open("trainKeys.json", 'w') as json_file:
                json_file.write(trainJson)
    def generateDataset(self, originalDict):
        kFold = 10
        valSize = len(originalDict)/(kFold+1)
        valSize = int(valSize)
        trainSize = len(originalDict) - valSize
        # 获取所有的键，并将它们随机打乱
        keys = list(originalDict.keys())
        random.shuffle(keys)
        # 计算分割点
        splitPoint = len(keys) // (kFold+1)
        # 分割键列表
        keys1 = keys[:splitPoint]
        keys2 = keys[splitPoint:]
        # 用分割后的键列表创建两个新字典
        #valData = {k: originalDict[k] for k in keys1}
        #trainData = {k: originalDict[k] for k in keys2}
        return keys1, keys2
    def generateQuestion(self):
         for i in tqdm(range(self.jsonNum)):
            tmpFileNameVal = self.fileName+"_"+str(self.chunkSize)+"_"+str(self.chunkOverlap)+"_val_"+"question.json"
            tmpFileNameTrain = self.fileName+"_"+str(self.chunkSize)+"_"+str(self.chunkOverlap)+"_train_"+"question.json"
            if not os.path.isfile(tmpFileNameVal):
                trainData = {}
                valData = {}
                for key in self.trainKeys:
                    [firstPos, secPos] = key.split("_")
                    firstPos = int(firstPos)
                    trainData[key] = self.allChunksMap[firstPos][secPos]
                for key in self.valKeys:
                    [firstPos, secPos] = key.split("_")
                    firstPos = int(firstPos)
                    valData[key] = self.allChunksMap[firstPos][secPos]
                self.trainQuestions  = generateQuestion(trainData)
                self.valQuestions  = generateQuestion(valData)
                valJson = json.dumps(self.valQuestions)
                with open(tmpFileNameVal, 'w') as json_file:
                    json_file.write(valJson)
                trainJson = json.dumps(self.trainQuestions)
                with open(tmpFileNameTrain, 'w') as json_file:
                    json_file.write(trainJson)
            else:
                with open(tmpFileNameTrain, 'r') as json_file:
                    self.trainQuestions = json.load(json_file)
                with open(tmpFileNameVal, 'r') as json_file:
                    self.valQuestions = json.load(json_file)

In [103]:
chunkSize = 512
chunkOverlap  = 128
oriFilePath = 'rfc7296.txt'
chunkDataSet = chunksData(oriFilePath, chunkSize, chunkOverlap)
chunkDataSet.splitTrainAndVal()
chunkDataSet.generateQuestion()

  0%|                                                                               | 1/2516 [00:02<1:44:01,  2.48s/it]

new******* ['What is the critical flag used for in the context of IKE SA?', 'How does the simultaneous IKE SA rekeying process work?'] 




  0%|                                                                               | 2/2516 [00:04<1:43:28,  2.47s/it]

new******* ['What is the order of keys used in ESP and AH protocols for encryption and integrity?', "Can you explain the significance of describing the order of keys in the protocol's specification?"] 




  0%|                                                                               | 3/2516 [00:07<1:45:44,  2.52s/it]

...

new******* ['What are the three main objectives of the provided context information?', "How does the proposed change in IKE's latency affect the initial exchange?"] 




100%|███████████████████████████████████████████████████████████████████████████████▋| 248/249 [12:47<00:03,  3.08s/it]

new******* ['What are the different MODP group sizes mentioned in the context information?', 'What purpose does the Diffie-Hellman exchange serve in the CREATE_CHILD_SA exchange?'] 




100%|████████████████████████████████████████████████████████████████████████████████| 3/3 [2:21:47<00:00, 2835.88s/it]

new******* ['What is the purpose of incorporating Ni and SPIi in the hash?', 'How does incorporating Ni and SPIi in the hash prevent an attacker from forging a message 3?'] 



In [14]:
model = SentenceTransformer("BAAI/bge-small-en")

In [107]:
#构造数据
trainExamples = []
for key in chunkDataSet.trainQuestions:
    [firstPos, secPos] = key.split("_")
    firstPos = int(firstPos)
    for i in range(len(chunkDataSet.trainQuestions[key])):
        trainExamples.append(InputExample(texts=[chunkDataSet.trainQuestions[key][i], chunkDataSet.allChunksMap[firstPos][secPos]]))
train_dataloader = DataLoader(trainExamples, shuffle=True, batch_size=16)
train_loss = losses.MultipleNegativesRankingLoss(model)

In [112]:
"""
word_embedding_model = model._first_module()

tokens = ["IKE_INIT", "IKE_AUTH", "SA", "IKE"]
word_embedding_model.tokenizer.add_tokens(tokens, special_tokens=False)
word_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))
"""

'\nword_embedding_model = model._first_module()\n\ntokens = ["IKE_INIT", "IKE_AUTH", "SA", "IKE"]\nword_embedding_model.tokenizer.add_tokens(tokens, special_tokens=False)\nword_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))\n'

In [ ]:
#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/315 [00:00<?, ?it/s]

In [ ]:
model.save("bge-small-en-FT-2")

In [ ]:
# function for hit rate evals
def evaluate(questions, model, chunksEmbeddings,index, top_k=5):
    hits = []
    print("start evaluate")
    for key in tqdm(questions):
        for i in range(len(questions[key])):
            questionEmbedding = model.encode(questions[key][i])
            D, I = index.search(questionEmbedding.reshape(1, -1), top_k)
            if int(key) in I[0]:
                hits.append(1)
            else:
                hits.append(0)
    return hits

In [ ]:
model = SentenceTransformer("bge-small-en-FT-2")

In [ ]:
#获取embedding
embeddings = {}
for key in tqdm(chunksMap):
    embeddings[key] = model.encode(chunksMap[key])
for key in tqdm(chunksMap):
    embeddings[key] = list(embeddings[key])

In [ ]:
index = faiss.IndexFlatL2(len(embeddings['0']))
index.add(np.array(list(embeddings.values())))

In [ ]:
hits = evaluate(valQuestions,model,embeddings,index)
print(sum(hits)/len(hits))

In [ ]:
hits = evaluate(trainQuestions,model,embeddings,index)
print(sum(hits)/len(hits))

In [ ]:
query = "what is the process of IKE_AUTH?"
questionEmbedding = model.encode(query)
D, I = index.search(questionEmbedding.reshape(1, -1), 5)
for i in range(len(I[0])):
    print("****************")
    print(chunksMap[str(I[0][i])])

In [33]:
####
#原来的模型
model = SentenceTransformer("BAAI/bge-small-en")
#获取embedding
embeddings = {}
for key in tqdm(chunksMap):
    embeddings[key] = model.encode(chunksMap[key])
for key in tqdm(chunksMap):
    embeddings[key] = list(embeddings[key])
index = faiss.IndexFlatL2(len(embeddings['0']))
index.add(np.array(list(embeddings.values())))

100%|█████████████████████████████████████████████████████████████████████████████| 922/922 [00:00<00:00, 71161.85it/s]


In [34]:
hits = evaluate(valQuestions,model,embeddings,index)
print(sum(hits)/len(hits))

start evaluate


100%|██████████████████████████████████████████████████████████████████████████████████| 83/83 [00:02<00:00, 35.87it/s]

0.6506024096385542


In [35]:
hits = evaluate(trainQuestions,model,embeddings,index)
print(sum(hits)/len(hits))

start evaluate


100%|████████████████████████████████████████████████████████████████████████████████| 839/839 [00:23<00:00, 35.09it/s]

0.66626936829559


In [37]:
query = "what is the process of IKE_AUTH?"
questionEmbedding = model.encode(query)
D, I = index.search(questionEmbedding.reshape(1, -1), 5)
for i in range(len(I[0])):
    print("****************")
    print(chunksMap[str(I[0][i])])

****************
URCE_IP),
                            N(NAT_DETECTION_DESTINATION_IP),]
                           [[N(HTTP_CERT_LOOKUP_SUPPORTED),] CERTREQ+,]
                           [V+][N+]

   cookie response     <-- N(COOKIE),
                           [V+][N+]

   different Diffie-   <-- N(INVALID_KE_PAYLOAD),
   Hellman group           [V+][N+]
   wanted

C.2.  IKE_AUTH Exchange without EAP

   request             --> IDi, [CERT+,]
                           [N(INITIAL_CONTACT),]
                           [[N(H
****************
17
       See Sections 1.2 and 1.3.

   AUTHENTICATION_FAILED                    24
       Sent in the response to an IKE_AUTH message when, for some
       reason, the authentication failed.  There is no associated
       data.  See also Section 2.21.2.

   SINGLE_PAIR_REQUIRED                     34
       See Section 2.9.

   NO_ADDITIONAL_SAS                        35
       See Section 1.3.

   INTERNAL_ADDRESS_FAILURE                 36
      